In [9]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext jupyternotify
%autonotify -a 30

<IPython.core.display.Javascript object>

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [24]:
from integrado import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
tokens=Tokens().tokenize("data/geo-mx-2004_min.json",replace=True,showProgressEach=10000)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 19:59:21.160513 :: 0.000166 - Processing item #0 
** Processed 1000 lines. Saved to /home/jovyan/public/B6_ProyectoIntegrador/data/geo-mx-2004_min.npy.


In [47]:
index_=Index().computePostingLists(tokens,showProgressEach=10000)
len(index_.postlists.keys())

** Counting TF & DF ...
2020-05-12 19:59:22.176658 :: 0.002929 - Processing item #0 
** Calculating TF-IDF...
2020-05-12 19:59:22.245640 :: 0.071911 - Processing item #0 
** Sorting posting lists...
2020-05-12 19:59:22.398810 :: 0.225081 - Processing item #0 


2435

In [48]:
ds=index_.o("covid coronavirus pandemia cuarentena",tokenize=True)
print(len(ds),ds[:5])
docs=tokens.getToks(ds)
print(len(docs),docs[:5])
[tw['text'] for tw in tokens.getTws(ds[:5])]

33 [12, 39, 42, 104, 143]
33 [['escucha', 'enlazados', 'informarse', 'pandemia', 'retransmision'], ['alturas', 'cuarentena', 'cuestiono', 'si', 'amigos'], ['si', 'sacrificar', 'colonia', 'culiacan', 'covid'], ['dice', 'vayamos', 'acabe', 'covid', 'da'], ['abril', 'mes', 'clave', 'tema', 'covid']]


['#QuédateEnCasa \n#FelizMiércoles\nEscucha #UnidosPorLasAudienciasCovid19 enlazados a \n@RadioEducacion para informarse  de la pandemia; y la retransmisión de los conciertos de la Orquesta Sinfónica Nacional en honor a Beethoven en la programación del 1 de #abril de 2020🐬 https://t.co/EBgx6FdsOY',
 'A estas alturas de la cuarentena #COVID19mx me cuestiono si mis "amigos" son muy positivos o muy pendejos;\nLo que comprobé es que soy muy tolerante',
 'Si tuvieras que sacrificar una colonia de Culiacán por el Covid-19 ¿Cual sería? y ¿Por qué Barrancos?',
 '@majoxmendez Dice que vayamos cuando acabe lo del Covid, que él da la casa y las klosters',
 '01 de abril del 2020 \nUn mes clave para el tema del COVID19\nCuídense mucho y dentro de sus posibilidades quédense en casa por favor 🙌']

In [54]:
from gensim.models import FastText

In [53]:
class Embeddings():
    
    def __init__(self,tokens):        
        model = FastText(size=100, window=3, min_count=1, min_n=2, max_n=5)
        model.build_vocab(sentences=tokens)
        model.train(sentences=tokens,total_examples=len(tokens), epochs=10)
        self.embeddings_=np.array(
            [np.mean([model.wv[tok] for tok in tw],axis=0) 
                   for tw in tokens])
        del(model)
        gc.collect()
        return self

In [56]:
embs=Embeddings(tokens)
print(type(embs), len(embs))
embs

TypeError: 'Tokens' object is not iterable

In [51]:
from skeleton import Clustering
class Clustering2(Clustering):
    
    # se asigna el método de inicialización de Kmeans (default="fft")
    def __init__(self, *args, k_init="fft", **kwargs):
        super().__init__(*args, **kwargs)
        self.k_init_ = k_init
        
    # asigna los elementos en la colección a su centroide más cercano
    # genera las etiquetas de los clusters
    # almacena la distancia de cada elemento a su centroide más cercano
    def _assign_nearest_centroids(self):
        self.labels_=[-1 for _ in self.data]
        self.dists_=[-1 for _ in self.data]
        for j in range(len(self.data)):
            dists = [(self.distance_function(c, self.data[j]), i, self.data[j])
                     for i, c in self.centroids_.items()]
            dists.sort()
            self.labels_[j]=dists[0][1]
            self.dists_[j]=(dists[0][0],j)
            
    def KMeans(self):
        self.centroids_ = {}
        C = []
        if self.k_init_ == 'fft':
            self.FFTraversal()
            for i, c in self.centroids_.items():
                C.append(c)
        elif self.k_init_ == 'random':
            C = np.random.uniform(low=np.min(self.data, axis=0) * .4,
                                  high=np.max(self.data, axis=0) * .4,
                                  size=(self.n_clusters, self.data.shape[1]))
#             self.labels_ = []
#             for i in range(len(self.data)):
#                 self.labels_.append(np.int64(np.random.choice(range(self.n_clusters))))
#             for i in range(self.n_clusters):
#                 cs = self.data[np.where(self.labels_ == np.int64(i))[0]]
#                 if len(cs) > 0:
#                     C.append(np.mean(cs, axis=0))
        else:
            raise Exception(f"k_init={self.k_init_} is not defined")
            
        C = np.array(C)
        error = np.inf
        while error != 0:
            self.labels_ = []
            C_old = deepcopy(C)
            for i in range(len(self.data)):
                distances = [(self.distance_function(c, self.data[i]), j, self.data[i])
                    for j, c in enumerate(C)]
                distances.sort()
                cluster = distances[0][1]
                self.labels_.append(cluster)
            for i in range(self.n_clusters):
                cs = self.data[np.where(self.labels_ == np.int64(i))[0]]
                if len(cs) > 0:
                    C[i] = np.mean(cs, axis=0)
            error = np.sum(C - C_old)
        self.centroids_ = {}
        for i, c in enumerate(C):
            self.centroids_[np.int32(i)] = c
        self._inertia()
        return self

    def FFTraversal(self):
        self.centroids_ = {}
        c1_idx = np.random.choice(self.data.shape[0])
        self.centroids_[np.int32(c1_idx)] = self.data[c1_idx, :]
        while len(self.centroids_.keys()) < self.n_clusters:
            self._assign_nearest_centroids()
            self.dists_.sort()
            cn_idx=self.dists_[-1][1]
            self.centroids_[np.int32(cn_idx)] = self.data[cn_idx, :]
        self._assign_nearest_centroids()
#         self.dists_.sort()
        self._inertia()

In [7]:
gc.collect()
import sys
local_vars = list(locals().items())
tot=0
vars_=[]
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    vars_.append((mem,var))
print("Total =",tot)
vars_.sort(reverse=True)
print(vars_)

20

Total = 21002
[(5688, 'ds'), (5504, 'docs'), (1064, 'Tokens'), (1064, 'InteractiveShell'), (1064, 'Index'), (1064, 'FastText'), (1064, 'Counter'), (400, 'defaultdict'), (278, '_i7'), (248, '_oh'), (248, 'Out'), (195, '_i1'), (172, '_i6'), (172, '_i'), (167, '_i2'), (144, 'tweet_iterator'), (138, '_iii'), (138, '_i4'), (136, '_ih'), (136, 'In'), (133, '_ii'), (133, '_i5'), (113, '__doc__'), (88, 'tb'), (88, 'sys'), (88, 'pd'), (88, 'os'), (88, 'np'), (88, 'gc'), (88, 'datetime'), (88, '__builtins__'), (88, '__builtin__'), (80, '_dh'), (72, 'get_ipython'), (72, '_i3'), (64, 'tokens'), (64, 'quit'), (64, 'index_'), (64, 'exit'), (57, '__name__'), (53, '___'), (53, '__'), (28, '_7'), (28, '_'), (16, '__spec__'), (16, '__package__'), (16, '__loader__')]
